In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore


In [3]:
faccount_df = pd.read_csv("./dataset/faccount.txt")
ftxn2_df = pd.read_csv("./dataset/ftxn2.txt")

In [4]:
ftxn2_df.head()

,TXN_ID,TXTYPE,AMOUNT,FROMACCTID,TOACCTID,TXDATE,REFERENCE,ISFRAUD,ISFLAGGED
0,950060711303,PAYMENT,320,500607,800242,2020-01-22T17:00:00,Dinner with Shane,N,Y
1,950060711401,PAYMENT,48,500607,800111,2020-01-23T08:00:00,Breakfast with Steven,N,Y
2,950060711402,TRANSFER,6209,500607,500929,2020-01-23T13:00:00,Holiday Cost,N,Y
3,950060711403,PAYMENT,66,500607,800252,2020-01-23T17:00:00,Meal with Mary,N,Y
4,950060711501,PAYMENT,47,500607,800116,2020-01-24T08:00:00,Coffee with Brian,N,Y


In [5]:
print(faccount_df.columns)
print(ftxn2_df.columns)

Index(['ACCTID', 'ACCTTYPE', 'ACCTNAME', 'FIRSTNAME', 'SURNAME', 'DOB',
       'ACCTCREATED', 'ACCTRISK', 'ADDRESS', 'ADDRESS_LAT', 'ADDRESS_LON',
       'CATEGORY'],
      dtype='object')
Index(['TXN_ID', 'TXTYPE', 'AMOUNT', 'FROMACCTID', 'TOACCTID', 'TXDATE',
       'REFERENCE', 'ISFRAUD', 'ISFLAGGED'],
      dtype='object')


In [6]:
#account types
print(faccount_df["ACCTTYPE"].unique())

#account categories
print(faccount_df["CATEGORY"].unique())


['BUSINESS' 'CREDIT' 'MAXI-SAVER' 'SAVINGS']
['Supermarket' 'Grocery' 'Cafe' 'Bakery' 'Hotel' 'Restaurant' 'Builders'
 'Lawyers' 'Crash Repairs' 'Apparel' 'Smallgoods' 'Property' 'Cleaning'
 'Gardening' 'Electrical' 'Wholesale' 'Events' 'Computers' 'Clinic'
 'Winery' 'Roasters' 'Pottery' 'Therapeutics' 'Fitness' 'Physiotherapy'
 'Beauty' 'Podiatry' 'Florist' 'Finance' 'Sports' 'Brewery' 'Used Cars'
 'Garage' 'Pharmacy' 'Butcher' 'Consulting' 'Distillery' 'Cheeses'
 'Management' 'Services' 'Childcare' 'Digital' 'Accounting' 'Salamis'
 'Trading' 'Music' 'CUSTOMER']


In [7]:
faccount_engineered = pd.DataFrame([])

faccount_engineered['ACCTID'] = faccount_df['ACCTID']

#add the person's full name 
faccount_engineered['FULLNAME'] = faccount_df['FIRSTNAME']+" "+faccount_df['SURNAME']

#add account type id, id generated by the order of unique accttypes
accttype = faccount_df["ACCTTYPE"].unique()
categorytype = faccount_df["CATEGORY"].unique()

def return_id(dataframe, array_to_search):
   return np.where(dataframe==array_to_search)

faccount_engineered['ACCTTYPE'] = faccount_df["ACCTTYPE"].astype('category').cat.codes

faccount_engineered['CATEGORY'] = faccount_df["CATEGORY"].astype('category').cat.codes


#count the number of time a person created an account
faccount_engineered['same_person_count'] = faccount_engineered.groupby('FULLNAME')['FULLNAME'].transform('count')

faccount_engineered[faccount_engineered['ACCTID']==800242].head()


,ACCTID,FULLNAME,ACCTTYPE,CATEGORY,same_person_count
280,800242,Taylor Marshall,1,35,1


In [17]:
from datetime import datetime

joined_features = ftxn2_df.merge(faccount_engineered, left_on='FROMACCTID', right_on='ACCTID')

joined_features = joined_features.drop(columns=['ISFRAUD', 'ISFLAGGED', 'ACCTID', 'FULLNAME'])

joined_features = joined_features.rename(columns={'ACCTTYPE':'ACCTTYPEFROM', 'CATEGORY':'CATEGORYFROM'})

joined_features = joined_features.merge(faccount_engineered, left_on='TOACCTID', right_on='ACCTID')

joined_features = joined_features.drop(columns=['ACCTID', 'FULLNAME'])

joined_features = joined_features.rename(columns={'ACCTTYPE':'ACCTTYPETO', 'CATEGORY':'CATEGORYTO'})

joined_features['TXTYPE'] = joined_features["TXTYPE"].astype('category').cat.codes

joined_features = joined_features.drop(columns=['TXN_ID', 'REFERENCE'])


def reformat_datetime(datetime_str):
    timestamp_str = datetime_str.replace('T', ' ')
    timestamp_datetime = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')
    timestamp_unix = timestamp_datetime.timestamp()

    return timestamp_unix

joined_features['timestamp'] = np.vectorize(reformat_datetime)(joined_features['TXDATE'])

joined_features = joined_features.drop(columns=['TXDATE'])

joined_features.head()

,TXTYPE,AMOUNT,FROMACCTID,TOACCTID,ACCTTYPEFROM,CATEGORYFROM,same_person_count_x,ACCTTYPETO,CATEGORYTO,same_person_count_y,timestamp
0,0,320,500607,800242,2,7,1,1,35,1,1.579712e+09
1,0,187,500607,800242,2,7,1,1,35,1,1.590772e+09
2,0,230,500607,800242,2,7,1,1,35,1,1.571504e+09
3,0,166,500607,800242,2,7,1,1,35,1,1.573751e+09
4,0,487,500607,800242,2,7,1,1,35,1,1.575565e+09


In [21]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()

timestamp_vals = joined_features['timestamp'].values.reshape(-1,1)
timestamp_scaled = min_max_scaler.fit_transform(timestamp_vals)
joined_features['timestamp_normed'] = timestamp_scaled[:,0]


final_joined_features = joined_features

final_joined_features = final_joined_features.drop(columns=['timestamp'])
final_joined_features.head(10)



,TXTYPE,AMOUNT,FROMACCTID,TOACCTID,ACCTTYPEFROM,CATEGORYFROM,same_person_count_x,ACCTTYPETO,CATEGORYTO,same_person_count_y,timestamp_normed
0,0,320,500607,800242,2,7,1,1,35,1,0.308405
1,0,187,500607,800242,2,7,1,1,35,1,0.659691
2,0,230,500607,800242,2,7,1,1,35,1,0.047684
3,0,166,500607,800242,2,7,1,1,35,1,0.119039
4,0,487,500607,800242,2,7,1,1,35,1,0.176672
5,0,342,500607,800242,2,7,1,1,35,1,0.209605
6,0,164,500606,800242,1,7,1,1,35,1,0.415437
7,0,155,500606,800242,1,7,1,1,35,1,0.486792
8,0,459,500606,800242,1,7,1,1,35,1,0.187650
9,0,125,500604,800242,0,7,1,1,35,1,0.250772


In [22]:
final_joined_features.to_csv('./dataset/joined_features.txt', index=False)